<h1>PAGERANK</h1>

In [1]:
import numpy as np
import pandas as pd
import nltk as nltk
import math
import operator
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/paz/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
data_set = pd.read_csv('../data/soc-sign-bitcoinotc.csv')
data_set.columns = ['source', 'target', 'rating', 'time']
data_set.head()


,source,target,rating,time
0,6,5,2,1.289242e+09
1,1,15,1,1.289243e+09
2,4,3,7,1.289245e+09
3,13,16,8,1.289254e+09
4,13,10,8,1.289254e+09


In [102]:
#Filtering The ones with rating grater or iqual to 8

new_data = data_set.loc[data_set['rating'] >= 8]
new_data.index = np.arange(0, len(new_data))
new_data



,source,target,rating,time
0,13,16,8,1.289254e+09
1,13,10,8,1.289254e+09
2,21,1,8,1.289441e+09
3,21,10,8,1.289441e+09
4,21,8,9,1.289441e+09
5,10,1,8,1.289556e+09
6,10,21,8,1.289556e+09
7,10,25,10,1.289556e+09
8,13,1,8,1.290521e+09
9,1,17,9,1.290969e+09


In [64]:
#Mapping sorce and target to a list

matter_inverstors = list(set(new_data.source).union(set(new_data.target)))


print(matter_inverstors)

[1, 2, 3, 4, 1366, 6, 7, 8, 9, 10, 4107, 13, 2063, 4112, 17, 19, 20, 21, 2070, 23, 4121, 2076, 29, 2078, 2079, 2080, 33, 2082, 35, 39, 4136, 41, 4141, 4144, 51, 4150, 57, 60, 61, 62, 64, 4163, 4164, 2118, 71, 4168, 3084, 4172, 2125, 2127, 80, 81, 2132, 4181, 88, 93, 2142, 1381, 2144, 16, 1708, 101, 4198, 104, 4205, 2158, 111, 113, 2067, 4212, 2165, 119, 4217, 4219, 2173, 2174, 127, 2176, 5824, 4226, 132, 134, 135, 4235, 2674, 4237, 4238, 143, 144, 4119, 149, 2198, 2073, 2200, 159, 160, 4257, 710, 2214, 167, 4266, 178, 2419, 2230, 184, 4281, 2240, 2241, 4098, 4291, 197, 198, 200, 201, 202, 4301, 4304, 4308, 2262, 215, 2266, 219, 223, 224, 4322, 3451, 230, 2087, 2284, 237, 241, 242, 245, 2296, 249, 4348, 2303, 256, 2305, 4354, 4356, 2311, 2313, 4365, 270, 4367, 4369, 278, 279, 280, 283, 284, 2342, 2344, 732, 2347, 300, 2349, 304, 309, 313, 4415, 2370, 4421, 2377, 330, 1643, 5859, 2388, 342, 3813, 353, 2404, 2409, 4458, 363, 2451, 2110, 1769, 2427, 4482, 2437, 2438, 395, 397, 4499, 475, 3

In [103]:
adj_dict = dict()
def fill_adj(adj_dict, data_set, matter_inverstors):
    for index in range(len(matter_inverstors)):
        element = matter_inverstors[index]
        print('index: ', index,'element: ', element)
        if (element in data_set.source):
            source = data_set.loc[index, 'source']
            target = data_set.loc[index, 'target']
            print(source, target)
            source_index = matter_inverstors.index(source)
            target_index = matter_inverstors.index(target)
            add_to_adj(adj_dict, source_index, target_index)
            print('source index', source_index)
        else:
            if (index not in adj_dict):
                adj_dict[index] = []
            else:
                print("HEREEEEE")

def add_to_adj(adj, source, target):
    if (source in adj):
        adj[source].append(target)
    else:
        adj[source] = [target]
    
    if (target not in adj):
        adj[target] = []
        
fill_adj(adj_dict, new_data, matter_inverstors)

print(sorted(adj_dict.keys()))


('index: ', 0, 'element: ', 1)
(13, 16)
('source index', 11)
('index: ', 1, 'element: ', 2)
(13, 10)
('source index', 11)
('index: ', 2, 'element: ', 3)
(21, 1)
('source index', 17)
('index: ', 3, 'element: ', 4)
(21, 10)
('source index', 17)
('index: ', 4, 'element: ', 1366)
('index: ', 5, 'element: ', 6)
(10, 1)
('source index', 9)
('index: ', 6, 'element: ', 7)
(10, 21)
('source index', 9)
('index: ', 7, 'element: ', 8)
(10, 25)
('source index', 9)
('index: ', 8, 'element: ', 9)
(13, 1)
('source index', 11)
('index: ', 9, 'element: ', 10)
(1, 17)
('source index', 0)
('index: ', 10, 'element: ', 4107)
('index: ', 11, 'element: ', 13)
(61, 20)
('source index', 38)
('index: ', 12, 'element: ', 2063)
('index: ', 13, 'element: ', 4112)
('index: ', 14, 'element: ', 17)
(119, 127)
('source index', 71)
('index: ', 15, 'element: ', 19)
(1, 2)
('source index', 0)
('index: ', 16, 'element: ', 20)
(127, 119)
('source index', 76)
('index: ', 17, 'element: ', 21)
(2, 3)
('source index', 1)
('inde

In [4]:
adj_dict = gen_adjacence_dict(data_set)
total_nodes = max(adj_dict.keys())


transition_matrix = np.zeros((total_nodes + 1, total_nodes + 1))

def fill_transition_matrix(transition_matrix, adj_dict):
    for node in adj_dict:
        adjacents = adj_dict[node]
        if (len(adjacents) > 0):
            weight = 1.0/len(adjacents)
            for adj in adjacents:
                transition_matrix[node][adj] = weight

fill_transition_matrix(transition_matrix, adj_dict)


In [ ]:
#Constants


In [5]:
dumping_matrix_factor = np.ones((total_nodes + 1, total_nodes + 1))
dumping_matrix_factor = (1.0/6.0 * np.matrix(dumping_matrix_factor))


In [6]:
m = 0.85 * transition_matrix + 0.15 * dumping_matrix_factor

In [7]:
normalized_vector = np.ones((total_nodes + 1, 1))
normalized_vector = (1.0/6.0 * np.matrix(normalized_vector))

In [8]:
cont = 0

def pagerank(v):
    global cont
    print(cont)
    calc = sum(abs(m * v - v))
    if (calc > 0.001):
        cont += 1
        return pagerank(m * v)
    else:
        cont += 1
        return m * v
    
    

In [9]:
v = normalized_vector
print(sum(m*v - v))
result = pagerank(v)

print(result)

[[149096.61666667]]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
[[inf]
 [inf]
 [inf]
 ...
 [inf]
 [inf]
 [inf]]


/home/paz/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in add
  
/home/paz/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in subtract
  
